# Standardization Utility

### Overview
This is a semi-automated process that can help in standardizing similar values of conformed attributes (dimensions) pulled from two different sources. It works under the following circumstances:
- Attribute values come from two sources.
- Attribute values are mainly lists.

### Process
- Loading and preprocessing
- Matching
    - the values are split on punctuation (`,`, `-`, `&`) into the individial names which are then exploded and used for joining the sources.
    - then values that contain >= 2 common individual names are considered as valid matches.
    - union of the two values are created from the valid matches.
    - matches are exported as a dictionary mapping the value and the union for each source.
- Other unmatched values are then outlined for further review.

### Notes
- This is a semi automatic utility, so manual review is necessary in every stage of th process to point out minor nuances that prevent values from being matched (.e.g plural vs singular forms, `ing` forms)

In [44]:
import requests
import pandas as pd
from dz_jobs_aggregator.utils import french_titlecase
import re

pd.set_option("max_colwidth", 600)
LANG = "fr"

In [45]:
# ---- Emploi Partner Transformations ----
emploi_partner_dimenion_url = "https://api-v4.emploipartner.com/api/params_functions?&pagination=false"
response = requests.get(emploi_partner_dimenion_url, headers={"Accept-Language": LANG})
label_key = "name"
ep_df = pd.DataFrame(response.json()["hydra:member"])[[label_key]].drop_duplicates()

# Replace any slashes with commas.
ep_df[label_key] = ep_df[label_key].apply(lambda x: re.sub(r"\s*/\s*", ", ", x))

# Apply custom titlecase
ep_df[label_key] = ep_df[label_key].apply(french_titlecase)

ep_df = ep_df.rename(columns={label_key: "emploi_partner"})

# ---- Emploitic Transformations ----
emploitic_dimension_url = f"https://emploitic.com/api/v4/lists/metiers?&lang={LANG}&per_page=30"
response = requests.get(emploitic_dimension_url)
em_label_key = "label"
em_df = pd.DataFrame(response.json()["collection"])[[em_label_key]].drop_duplicates()

# Apply custom titlecase
em_df[em_label_key] = em_df[em_label_key].apply(french_titlecase)
em_df = em_df.rename(columns={em_label_key: "emploitic"})

# ---- Merge Both Data Sources ----
merged = pd.merge(ep_df, em_df, left_on="emploi_partner", right_on="emploitic", how="outer")
# merged.to_json("merged_functions.json", orient="records", force_ascii=False)
merged

,emploi_partner,emploitic
0,"Accueil, Administration - Services Généraux",NaN
1,NaN,"Administration, Finance, Comptabilité & Juridique"
2,NaN,"Administration, Moyens Généraux"
3,"Armées, Défense",NaN
4,NaN,"Assistanat, Secrétariat"
...,...,...
61,"Tourisme, Hôtellerie, Restauration",NaN
62,"Transport, Achat, Logistique, Emballage",NaN
63,NaN,"Télécommunication, Réseaux"
64,"Télécommunications, Systèmes, Réseau",NaN


In [46]:
# split on (',', '-' or '&')
split_pattern = "\s*,\s*|\s*-\s*|\s+&\s*|\s*&\s+"
ep_df["emploi_partner_name"] = ep_df["emploi_partner"].apply(
    lambda x: re.split(split_pattern, x)
)
ep_df["emploi_partner_set"] = ep_df["emploi_partner_name"].copy().apply(lambda x: set(x))

em_df["emploitic_name"] = em_df["emploitic"].apply(
    lambda x: re.split(split_pattern, x)
)
em_df["emploitic_set"] = em_df["emploitic_name"].copy().apply(lambda x: set(x))

ep_df_expld = ep_df.explode("emploi_partner_name")
em_df_expld = em_df.explode("emploitic_name")
merged_expld = pd.merge(
    ep_df_expld,
    em_df_expld,
    left_on="emploi_partner_name",
    right_on="emploitic_name",
)
set_union = lambda row: row["emploi_partner_set"] | row["emploitic_set"]
set_to_str = lambda row: ', '.join(sorted(row["union"]))
merged_expld["union"] = merged_expld.apply(set_union, axis="columns")
merged_expld["union_str"] = merged_expld.apply(set_to_str, axis="columns")
merged_expld

,emploi_partner,emploi_partner_name,emploi_partner_set,emploitic,emploitic_name,emploitic_set,union,union_str
0,"Accueil, Administration - Services Généraux",Administration,"{Accueil, Services Généraux, Administration}","Administration, Finance, Comptabilité & Juridique",Administration,"{Finance, Comptabilité, Juridique, Administration}","{Finance, Accueil, Services Généraux, Comptabilité, Juridique, Administration}","Accueil, Administration, Comptabilité, Finance, Juridique, Services Généraux"
1,"Accueil, Administration - Services Généraux",Administration,"{Accueil, Services Généraux, Administration}","Administration, Moyens Généraux",Administration,"{Moyens Généraux, Administration}","{Accueil, Services Généraux, Moyens Généraux, Administration}","Accueil, Administration, Moyens Généraux, Services Généraux"
2,"Création, Culture, Spectacles",Création,"{Spectacles, Culture, Création}","Commercial, Marketing, Communication & Création",Création,"{Marketing, Commercial, Création, Communication}","{Création, Communication, Culture, Marketing, Commercial, Spectacles}","Commercial, Communication, Création, Culture, Marketing, Spectacles"
3,"Création, Culture, Spectacles",Création,"{Spectacles, Culture, Création}","Création, Design",Création,"{Création, Design}","{Création, Culture, Spectacles, Design}","Création, Culture, Design, Spectacles"
4,"Commerce, Distribution, Vente",Vente,"{Commerce, Distribution, Vente}","Vente, Télévente, Assistanat",Vente,"{Assistanat, Vente, Télévente}","{Commerce, Distribution, Télévente, Assistanat, Vente}","Assistanat, Commerce, Distribution, Télévente, Vente"
5,"Juridique, Fiscal",Juridique,"{Juridique, Fiscal}","Administration, Finance, Comptabilité & Juridique",Juridique,"{Finance, Comptabilité, Juridique, Administration}","{Finance, Fiscal, Comptabilité, Juridique, Administration}","Administration, Comptabilité, Finance, Fiscal, Juridique"
6,"Juridique, Fiscal",Juridique,"{Juridique, Fiscal}","Juridique, Fiscal, Audit, Conseil",Juridique,"{Audit, Conseil, Juridique, Fiscal}","{Conseil, Fiscal, Audit, Juridique}","Audit, Conseil, Fiscal, Juridique"
7,"Juridique, Fiscal",Fiscal,"{Juridique, Fiscal}","Juridique, Fiscal, Audit, Conseil",Fiscal,"{Audit, Conseil, Juridique, Fiscal}","{Conseil, Fiscal, Audit, Juridique}","Audit, Conseil, Fiscal, Juridique"
8,"Edition, Imprimerie, Presse, Traduction",Traduction,"{Presse, Traduction, Edition, Imprimerie}","Journalisme, Médias, Traduction",Traduction,"{Médias, Traduction, Journalisme}","{Traduction, Edition, Imprimerie, Presse, Médias, Journalisme}","Edition, Imprimerie, Journalisme, Médias, Presse, Traduction"
9,"Industrie, Electrique et Électrotechnique, Production, Maintenance",Industrie,"{Production, Electrique et Électrotechnique, Maintenance, Industrie}","Ingénierie, Industrie, Construction",Industrie,"{Construction, Ingénierie, Industrie}","{Production, Electrique et Électrotechnique, Ingénierie, Maintenance, Construction, Industrie}","Construction, Electrique et Électrotechnique, Industrie, Ingénierie, Maintenance, Production"


In [47]:
# get the rows where the two sources have 2 or more names in commun
# these are almost certainly to be valid standardizations
# other rows need further inspection
merged_filtered = merged_expld[
    merged_expld.duplicated(["emploi_partner", "emploitic"], keep=False)
].copy()
merged_filtered["replace_emploi_partner"] = merged_filtered["union"] != merged_filtered["emploi_partner_set"]
merged_filtered["replace_emploitic"] = merged_filtered["union"] != merged_filtered["emploitic_set"]
merged_filtered = merged_filtered.sort_values("emploi_partner")
merged_filtered

,emploi_partner,emploi_partner_name,emploi_partner_set,emploitic,emploitic_name,emploitic_set,union,union_str,replace_emploi_partner,replace_emploitic
19,"Création, Design, Médias Numériques",Création,"{Création, Design, Médias Numériques}","Création, Design",Création,"{Création, Design}","{Création, Médias Numériques, Design}","Création, Design, Médias Numériques",False,True
20,"Création, Design, Médias Numériques",Design,"{Création, Design, Médias Numériques}","Création, Design",Design,"{Création, Design}","{Création, Médias Numériques, Design}","Création, Design, Médias Numériques",False,True
10,"Industrie, Electrique et Électrotechnique, Production, Maintenance",Industrie,"{Production, Electrique et Électrotechnique, Maintenance, Industrie}","Production, Méthode, Industrie",Industrie,"{Production, Industrie, Méthode}","{Production, Electrique et Électrotechnique, Méthode, Maintenance, Industrie}","Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production",True,True
11,"Industrie, Electrique et Électrotechnique, Production, Maintenance",Production,"{Production, Electrique et Électrotechnique, Maintenance, Industrie}","Production, Méthode, Industrie",Production,"{Production, Industrie, Méthode}","{Production, Electrique et Électrotechnique, Méthode, Maintenance, Industrie}","Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production",True,True
16,"Informatique, Systèmes d'Information, Analyse et Science des Données",Informatique,"{Analyse et Science des Données, Systèmes d'Information, Informatique}","Informatique, Systèmes d'Information, Internet",Informatique,"{Systèmes d'Information, Informatique, Internet}","{Analyse et Science des Données, Systèmes d'Information, Informatique, Internet}","Analyse et Science des Données, Informatique, Internet, Systèmes d'Information",True,True
17,"Informatique, Systèmes d'Information, Analyse et Science des Données",Systèmes d'Information,"{Analyse et Science des Données, Systèmes d'Information, Informatique}","Informatique, Systèmes d'Information, Internet",Systèmes d'Information,"{Systèmes d'Information, Informatique, Internet}","{Analyse et Science des Données, Systèmes d'Information, Informatique, Internet}","Analyse et Science des Données, Informatique, Internet, Systèmes d'Information",True,True
38,"Ingénierie, Bureau d'Études, Projet, R&D",R&D,"{R&D, Bureau d'Études, Ingénierie, Projet}","Ingénierie, Etudes, Projet, R&D",R&D,"{R&D, Projet, Ingénierie, Etudes}","{Bureau d'Études, Ingénierie, Etudes, R&D, Projet}","Bureau d'Études, Etudes, Ingénierie, Projet, R&D",True,True
36,"Ingénierie, Bureau d'Études, Projet, R&D",Ingénierie,"{R&D, Bureau d'Études, Ingénierie, Projet}","Ingénierie, Etudes, Projet, R&D",Ingénierie,"{R&D, Projet, Ingénierie, Etudes}","{Bureau d'Études, Ingénierie, Etudes, R&D, Projet}","Bureau d'Études, Etudes, Ingénierie, Projet, R&D",True,True
37,"Ingénierie, Bureau d'Études, Projet, R&D",Projet,"{R&D, Bureau d'Études, Ingénierie, Projet}","Ingénierie, Etudes, Projet, R&D",Projet,"{R&D, Projet, Ingénierie, Etudes}","{Bureau d'Études, Ingénierie, Etudes, R&D, Projet}","Bureau d'Études, Etudes, Ingénierie, Projet, R&D",True,True
6,"Juridique, Fiscal",Juridique,"{Juridique, Fiscal}","Juridique, Fiscal, Audit, Conseil",Juridique,"{Audit, Conseil, Juridique, Fiscal}","{Conseil, Fiscal, Audit, Juridique}","Audit, Conseil, Fiscal, Juridique",True,False


In [48]:
# the replacements to be used in each source data transformation step
df_values_to_replace_ep = merged_filtered.loc[
    merged_filtered["replace_emploi_partner"], ["emploi_partner", "union_str"]
].drop_duplicates()
df_values_to_replace_em = merged_filtered.loc[
    merged_filtered["replace_emploitic"], ["emploitic", "union_str"]
].drop_duplicates()
display(df_values_to_replace_ep)
display(df_values_to_replace_em)

,emploi_partner,union_str
10,"Industrie, Electrique et Électrotechnique, Production, Maintenance","Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production"
16,"Informatique, Systèmes d'Information, Analyse et Science des Données","Analyse et Science des Données, Informatique, Internet, Systèmes d'Information"
38,"Ingénierie, Bureau d'Études, Projet, R&D","Bureau d'Études, Etudes, Ingénierie, Projet, R&D"
6,"Juridique, Fiscal","Audit, Conseil, Fiscal, Juridique"
23,"Marketing, Communication, RP","Commercial, Communication, Création, Marketing, RP"
29,"Tourisme, Hôtellerie, Restauration","Hôtellerie, Loisirs, Restauration, Tourisme"
32,"Transport, Achat, Logistique, Emballage","Achat, Emballage, Logistique, Stock, Transport"


,emploitic,union_str
19,"Création, Design","Création, Design, Médias Numériques"
10,"Production, Méthode, Industrie","Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production"
16,"Informatique, Systèmes d'Information, Internet","Analyse et Science des Données, Informatique, Internet, Systèmes d'Information"
38,"Ingénierie, Etudes, Projet, R&D","Bureau d'Études, Etudes, Ingénierie, Projet, R&D"
23,"Commercial, Marketing, Communication & Création","Commercial, Communication, Création, Marketing, RP"
22,"Marketing, Communication","Communication, Marketing, RP"
32,"Logistique, Achat, Stock, Transport","Achat, Emballage, Logistique, Stock, Transport"


In [49]:
values_to_replace_ep = dict(zip(
    df_values_to_replace_ep["emploi_partner"], df_values_to_replace_ep["union_str"]
))

values_to_replace_em = dict(zip(
    df_values_to_replace_em["emploitic"], df_values_to_replace_em["union_str"]
))
display(values_to_replace_ep)
display(values_to_replace_em)

{'Industrie, Electrique et Électrotechnique, Production, Maintenance': 'Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production',
 "Informatique, Systèmes d'Information, Analyse et Science des Données": "Analyse et Science des Données, Informatique, Internet, Systèmes d'Information",
 "Ingénierie, Bureau d'Études, Projet, R&D": "Bureau d'Études, Etudes, Ingénierie, Projet, R&D",
 'Juridique, Fiscal': 'Audit, Conseil, Fiscal, Juridique',
 'Marketing, Communication, RP': 'Commercial, Communication, Création, Marketing, RP',
 'Tourisme, Hôtellerie, Restauration': 'Hôtellerie, Loisirs, Restauration, Tourisme',
 'Transport, Achat, Logistique, Emballage': 'Achat, Emballage, Logistique, Stock, Transport'}

{'Création, Design': 'Création, Design, Médias Numériques',
 'Production, Méthode, Industrie': 'Electrique et Électrotechnique, Industrie, Maintenance, Méthode, Production',
 "Informatique, Systèmes d'Information, Internet": "Analyse et Science des Données, Informatique, Internet, Systèmes d'Information",
 'Ingénierie, Etudes, Projet, R&D': "Bureau d'Études, Etudes, Ingénierie, Projet, R&D",
 'Commercial, Marketing, Communication & Création': 'Commercial, Communication, Création, Marketing, RP',
 'Marketing, Communication': 'Communication, Marketing, RP',
 'Logistique, Achat, Stock, Transport': 'Achat, Emballage, Logistique, Stock, Transport'}

In [50]:
# inspect the remaining rows and look for matchable values
# that were missed due to subtle changes (e.g Réseaux vs Réseau)
unmatched_ep_df = ep_df[
    ~ep_df["emploi_partner"].isin(values_to_replace_ep.keys())
]
unmatched_em_df = em_df[
    ~em_df["emploitic"].isin(values_to_replace_em.keys())
]
merged_unmatched = pd.merge(
    unmatched_ep_df,
    unmatched_em_df,
    left_on="emploi_partner",
    right_on="emploitic",
    how="outer"
)
merged_unmatched[["emploi_partner", "emploitic"]]

,emploi_partner,emploitic
0,"Accueil, Administration - Services Généraux",NaN
1,NaN,"Administration, Finance, Comptabilité & Juridique"
2,NaN,"Administration, Moyens Généraux"
3,"Armées, Défense",NaN
4,NaN,"Assistanat, Secrétariat"
5,"Audit, Gestion, Finances, Compta",NaN
6,NaN,Autre
7,Autres,Autres
8,"Banque, Bourse, Assurance",NaN
9,"Batiment, Équipement",NaN
